# Starbucks Capstone Challenge

## Abstract

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [2]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

Matplotlib is building the font cache; this may take a moment.


## Data Cleaning

### Portfolio

In [29]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [60]:
# Available channels
channels = ["email", "mobile", "social", "web"]

# Create a one hot encoding for channel type
portfolioWithChannelsEncoded = portfolio.copy()
for channel in channels:
    portfolioWithChannelsEncoded[channel] = portfolio["channels"].apply(lambda cs: 1*(channel in cs))

portfolio_df = portfolioWithChannelsEncoded.drop(columns="channels")
portfolio_df

,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


### Profile

In [56]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [62]:
profile_df = profile.copy()
profile_df["became_member_on"] = pd.to_datetime(profile["became_member_on"].astype(str))
profile_df.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


### Transcript

In [64]:
transcript.head()


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [68]:
transcript['value'].apply(lambda d: d.keys()).explode().unique()

array(['offer id', 'amount', 'offer_id', 'reward'], dtype=object)

In [110]:
transcript_temp = transcript.copy()

# Create event number per person
transcript_temp = transcript_temp.sort_values(["person", "time"])
transcript_temp["event_no"] = transcript_temp.groupby("person").cumcount() + 1

# Explode value column to a list of key-values pairs
transcript_temp["aux"] = transcript_temp["value"].apply(lambda d: d.items())
transcript_temp = transcript_temp.explode("aux")

# Extract key and value for each exploded row
transcript_temp["k"] = transcript_temp["aux"].str[0]
transcript_temp["v"] = transcript_temp["aux"].str[1]

transcript_temp

,person,event,value,time,event_no,aux,k,v
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,1,"(offer id, 5a8bc65990b245e5a138643cd4eb9837)",offer id,5a8bc65990b245e5a138643cd4eb9837
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,2,"(offer id, 5a8bc65990b245e5a138643cd4eb9837)",offer id,5a8bc65990b245e5a138643cd4eb9837
89291,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},228,3,"(amount, 22.16)",amount,22.16
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,4,"(offer id, 3f207df678b143eea3cee63160fa8bed)",offer id,3f207df678b143eea3cee63160fa8bed
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,5,"(offer id, 3f207df678b143eea3cee63160fa8bed)",offer id,3f207df678b143eea3cee63160fa8bed
...,...,...,...,...,...,...,...,...
258362,ffff82501cea40309d5fdd7edcca4a07,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,576,30,"(offer_id, 2906b810c7d4411798c6938adc9daaa5)",offer_id,2906b810c7d4411798c6938adc9daaa5
258362,ffff82501cea40309d5fdd7edcca4a07,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,576,30,"(reward, 2)",reward,2
262475,ffff82501cea40309d5fdd7edcca4a07,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},582,31,"(offer id, 2906b810c7d4411798c6938adc9daaa5)",offer id,2906b810c7d4411798c6938adc9daaa5
274809,ffff82501cea40309d5fdd7edcca4a07,transaction,{'amount': 10.12},606,32,"(amount, 10.12)",amount,10.12


In [159]:
# Pivot
transcript_df = transcript_temp.pivot(
        index=["person", "event_no", "event", "time"],
        columns="k",
        values="v"
    )

# Reset index
transcript_df = transcript_df.reset_index()
transcript_df.columns.name = None

# Clean up values
transcript_df["offer_id"] = transcript_df["offer_id"].combine_first(transcript_df["offer id"])
transcript_df = transcript_df.drop(columns="offer id")
transcript_df["amount"] = pd.to_numeric(transcript_df["amount"])

# Fill amount and reward with 0 when nan for ease of manipulation
transcript_df["amount"] = transcript_df["amount"].fillna(0)
transcript_df["reward"] = transcript_df["reward"].fillna(0)

transcript_df

,person,event_no,event,time,amount,offer_id,reward
0,0009655768c64bdeb2e877511632db8f,1,offer received,168,0.00,5a8bc65990b245e5a138643cd4eb9837,0
1,0009655768c64bdeb2e877511632db8f,2,offer viewed,192,0.00,5a8bc65990b245e5a138643cd4eb9837,0
2,0009655768c64bdeb2e877511632db8f,3,transaction,228,22.16,NaN,0
3,0009655768c64bdeb2e877511632db8f,4,offer received,336,0.00,3f207df678b143eea3cee63160fa8bed,0
4,0009655768c64bdeb2e877511632db8f,5,offer viewed,372,0.00,3f207df678b143eea3cee63160fa8bed,0
...,...,...,...,...,...,...,...
306529,ffff82501cea40309d5fdd7edcca4a07,29,transaction,576,14.23,NaN,0
306530,ffff82501cea40309d5fdd7edcca4a07,30,offer completed,576,0.00,2906b810c7d4411798c6938adc9daaa5,2
306531,ffff82501cea40309d5fdd7edcca4a07,31,offer viewed,582,0.00,2906b810c7d4411798c6938adc9daaa5,0
306532,ffff82501cea40309d5fdd7edcca4a07,32,transaction,606,10.12,NaN,0


In [160]:
transcript_df.head(20)

,person,event_no,event,time,amount,offer_id,reward
0,0009655768c64bdeb2e877511632db8f,1,offer received,168,0.00,5a8bc65990b245e5a138643cd4eb9837,0
1,0009655768c64bdeb2e877511632db8f,2,offer viewed,192,0.00,5a8bc65990b245e5a138643cd4eb9837,0
2,0009655768c64bdeb2e877511632db8f,3,transaction,228,22.16,NaN,0
3,0009655768c64bdeb2e877511632db8f,4,offer received,336,0.00,3f207df678b143eea3cee63160fa8bed,0
4,0009655768c64bdeb2e877511632db8f,5,offer viewed,372,0.00,3f207df678b143eea3cee63160fa8bed,0
5,0009655768c64bdeb2e877511632db8f,6,offer received,408,0.00,f19421c1d4aa40978ebb69ca19b0e20d,0
6,0009655768c64bdeb2e877511632db8f,7,transaction,414,8.57,NaN,0
7,0009655768c64bdeb2e877511632db8f,8,offer completed,414,0.00,f19421c1d4aa40978ebb69ca19b0e20d,5
8,0009655768c64bdeb2e877511632db8f,9,offer viewed,456,0.00,f19421c1d4aa40978ebb69ca19b0e20d,0
9,0009655768c64bdeb2e877511632db8f,10,offer received,504,0.00,fafdcd668e3743c1bb461111dcafc2a4,0


## Feature Engineering

In [163]:
transcript_feats = transcript_df.copy()

# Create cumulative amount spent and cumulative amount rewarded (up to that point)
transcript_feats["cum_amount"] = transcript_feats.groupby("person")["amount"].transform(pd.Series.cumsum)
transcript_feats["cum_amount"] = transcript_feats["cum_amount"] - transcript_feats["amount"]
transcript_feats["cum_reward"] = transcript_feats.groupby("person")["reward"].transform(pd.Series.cumsum)
transcript_feats["cum_reward"] = transcript_feats["cum_reward"] - transcript_feats["reward"]

transcript_feats.head(20)

In [154]:
transcript_feats[transcript_feats["person"]=="ffff82501cea40309d5fdd7edcca4a07"]

,person,event_no,event,time,amount,offer_id,reward,cum_amount,cum_reward
306501,ffff82501cea40309d5fdd7edcca4a07,1,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.00,0.0
306502,ffff82501cea40309d5fdd7edcca4a07,2,offer viewed,6,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.00,0.0
306503,ffff82501cea40309d5fdd7edcca4a07,3,transaction,60,16.06,NaN,NaN,0.00,0.0
306504,ffff82501cea40309d5fdd7edcca4a07,4,offer completed,60,NaN,fafdcd668e3743c1bb461111dcafc2a4,2,0.00,0.0
306505,ffff82501cea40309d5fdd7edcca4a07,5,transaction,78,7.18,NaN,NaN,16.06,0.0
306506,ffff82501cea40309d5fdd7edcca4a07,6,transaction,120,9.12,NaN,NaN,23.24,0.0
306507,ffff82501cea40309d5fdd7edcca4a07,7,offer received,168,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.00,0.0
306508,ffff82501cea40309d5fdd7edcca4a07,8,offer viewed,174,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.00,0.0
306509,ffff82501cea40309d5fdd7edcca4a07,9,transaction,198,22.88,NaN,NaN,32.36,0.0
306510,ffff82501cea40309d5fdd7edcca4a07,10,offer completed,198,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0.00,2.0


In [155]:
# ...
aux = transcript_feats[transcript_feats["event"]=="offer received"]
aux.groupby(["person", "offer_id"]).size()

person                            offer_id                        
0009655768c64bdeb2e877511632db8f  2906b810c7d4411798c6938adc9daaa5    1
                                  3f207df678b143eea3cee63160fa8bed    1
                                  5a8bc65990b245e5a138643cd4eb9837    1
                                  f19421c1d4aa40978ebb69ca19b0e20d    1
                                  fafdcd668e3743c1bb461111dcafc2a4    1
                                                                     ..
fffad4f4828548d1b5583907f2e9906b  f19421c1d4aa40978ebb69ca19b0e20d    2
ffff82501cea40309d5fdd7edcca4a07  0b1e1539f2cc45b7b9fa7c272da2e1d7    1
                                  2906b810c7d4411798c6938adc9daaa5    3
                                  9b98b8c7a33c4b65b9aebfe6a799e6d9    1
                                  fafdcd668e3743c1bb461111dcafc2a4    1
Length: 63288, dtype: int64